# Preliminaries

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.utils import resample

#sys.path.append("/Users/paolo/Documents/methods/CMI_FS")
#from feature_selection import forwardFeatureSelection

sys.path.append("../LinCFA")
from LinCFA import LinCFA

sys.path.append("../NonLinCFA")
from NonLinCFA import NonLinCFA

sys.path.append("../GenLinCFA")
from GenLinCFA import GenLinCFA

sys.path.append("../droughts")
from aux import prepare_target,prepare_features,compare_methods

#from aux import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,aggregate_unfolded_data_onlyTrain,FS_with_linearWrapper,compare_methods, compute_r2


In [2]:
def compute_CI(list,n):
    print(f'{np.mean(list)} +- {1.96*np.std(list)/np.sqrt(n)}')

In [3]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn import utils
from sklearn.preprocessing import KernelCenterer, scale
from sklearn.metrics.pairwise import pairwise_kernels
from scipy import linalg
from scipy.sparse.linalg import eigsh as ssl_eigsh
from scipy.io import arff
from sklearn.impute import SimpleImputer

class spca(BaseEstimator, TransformerMixin):
    
    def __init__(self, num_components, kernel="linear", eigen_solver='auto', 
                 max_iterations=None, gamma=0, degree=3, coef0=1, alpha=1.0, 
                 tolerance=0, fit_inverse_transform=False):
        
        self._num_components = num_components
        self._gamma = gamma
        self._tolerance = tolerance
        self._fit_inverse_transform = fit_inverse_transform
        self._max_iterations = max_iterations
        self._degree = degree
        self._kernel = kernel
        self._eigen_solver = eigen_solver
        self._coef0 = coef0
        self._centerer = KernelCenterer()
        self._alpha = alpha
        self._alphas = []
        self._lambdas = []
        
        
    def _get_kernel(self, X, Y=None):
        # Returns a kernel matrix K such that K_{i, j} is the kernel between the ith and jth vectors 
        # of the given matrix X, if Y is None. 
        
        # If Y is not None, then K_{i, j} is the kernel between the ith array from X and the jth array from Y.
        
        # valid kernels are 'linear, rbf, poly, sigmoid, precomputed'
        
        args = {"gamma": self._gamma, "degree": self._degree, "coef0": self._coef0}
        
        return pairwise_kernels(X, Y, metric=self._kernel, n_jobs=-1, filter_params=True, **args)
    
    
    
    def _fit(self, X, Y):
        
        # calculate kernel matrix of the labels Y and centre it and call it K (=H.L.H)
        K = self._centerer.fit_transform(self._get_kernel(Y))
        
        # deciding on the number of components to use
        if self._num_components is not None:
            num_components = min(K.shape[0], self._num_components)
        else:
            num_components = self.K.shape[0]
        
        # Scale X
        # scaled_X = scale(X)
        
        # calculate the eigen values and eigen vectors for X^T.K.X
        Q = (X.T).dot(K).dot(X)
        
        # If n_components is much less than the number of training samples, 
        # arpack may be more efficient than the dense eigensolver.
        if (self._eigen_solver=='auto'):
            if (Q.shape[0]/num_components) > 20:
                eigen_solver = 'arpack'
            else:
                eigen_solver = 'dense'
        else:
            eigen_solver = self._eigen_solver
        
        if eigen_solver == 'dense':
            # Return the eigenvalues (in ascending order) and eigenvectors of a Hermitian or symmetric matrix.
            self._lambdas, self._alphas = linalg.eigh(Q, eigvals=(Q.shape[0] - num_components, Q.shape[0] - 1))
            # argument eigvals = Indexes of the smallest and largest (in ascending order) eigenvalues
        
        elif eigen_solver == 'arpack':
            # deprecated :: self._lambdas, self._alphas = utils.arpack.eigsh(A=Q, num_components, which="LA", tol=self._tolerance)
            self._lambdas, self._alphas = ssl_eigsh(A=Q, k=num_components, which="LA", tol=self._tolerance)
            
        indices = self._lambdas.argsort()[::-1]
        
        self._lambdas = self._lambdas[indices]
        self._lambdas = self._lambdas[self._lambdas > 0]  # selecting values only for non zero eigen values
        
        self._alphas = self._alphas[:, indices]
        #return self._alphas
        self._alphas = self._alphas[:, self._lambdas > 0]  # selecting values only for non zero eigen values
        
        self.X_fit = X

        
    def _transform(self):
        return self.X_fit.dot(self._alphas)
        
        
    def transform(self, X):
        return X.dot(self._alphas)
        
        
    def fit(self, X, Y):
        self._fit(X,Y)
        return
        
        
    def fit_and_transform(self, X, Y):
        self.fit(X, Y)
        return self._transform()
    

In [4]:
df = pd.read_csv('pd_speech_features.csv')
df = df.iloc[1:,2:]
for col in list(df.columns):
    df[col] = df.apply(lambda x: float(x[col]),axis=1)
normalized_df=(df-df.mean())/df.std()
normalized_df = normalized_df.dropna()
normalized_df['mean_std'] = df.iloc[:,-1]
normalized_df = normalized_df.drop(['Unnamed: 754'],axis=1)
normalized_df = normalized_df.sample(frac=1)
normalized_df = normalized_df.reset_index(drop=True)
normalized_df = pd.concat((normalized_df[normalized_df.mean_std==1].sample(192),normalized_df[normalized_df.mean_std==0])).sample(frac=1).reset_index(drop=True)
normalized_df

,Baseline Features,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 745,Unnamed: 746,Unnamed: 747,Unnamed: 748,Unnamed: 749,Unnamed: 750,Unnamed: 751,Unnamed: 752,Unnamed: 753,mean_std
0,0.451557,0.874179,1.045764,-1.330110,-1.324701,2.021293,-0.291851,0.907752,1.334685,0.800446,...,-0.573085,-0.633095,-0.552079,0.542206,-0.191752,-0.117083,1.796074,1.638597,1.662636,1.0
1,0.528524,1.394274,-0.611802,0.342956,0.345277,-0.524224,-0.461129,-0.340345,-0.393161,-0.351778,...,1.257259,1.120810,0.849854,0.093424,-0.276842,-0.164141,-0.213365,-0.068548,-0.728115,1.0
2,0.598226,1.735219,-0.912219,0.947679,0.948884,-0.957429,-0.472398,-0.587681,-0.572626,-0.514925,...,3.666007,2.964905,1.324396,0.785885,1.300541,1.411641,0.986800,0.199255,-0.811981,0.0
3,0.393138,-1.078687,0.337104,0.040595,0.043474,-0.256398,-0.455082,-0.405033,-0.407570,-0.412958,...,1.961333,1.542189,1.420803,1.317547,0.887110,0.120151,-0.583573,-0.693603,-0.768693,0.0
4,0.400049,-0.501792,-1.343454,-0.554049,-0.550072,0.438930,-0.465801,-0.336540,-0.280940,-0.290598,...,-0.564682,1.852167,1.506109,1.486124,1.930102,1.731749,1.048877,0.814499,-0.664080,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,0.540929,0.426806,1.584899,-2.438768,-2.501734,0.223554,1.997729,3.373504,2.579154,2.860174,...,-0.483653,-0.519788,-0.563744,-0.491350,-0.509392,-0.586968,-0.696437,-0.670822,-0.593284,1.0
380,0.082199,-0.151811,-0.647017,0.272405,0.274857,-0.470949,-0.430070,-0.317514,-0.373511,-0.433352,...,1.006034,1.375266,1.972268,2.657154,2.573008,2.584239,2.349789,2.224907,1.823029,1.0
381,0.451557,-0.697724,-1.806920,1.229882,1.230567,-1.112991,-0.477895,-0.690421,-0.634195,-0.545515,...,-0.263441,-0.604409,-0.629514,-0.563369,-0.535917,-0.582672,-0.729918,-0.724366,-0.825401,0.0
382,0.132644,1.235205,-0.830294,0.987993,0.989124,-0.981772,-0.440927,-0.530603,-0.545554,-0.504728,...,2.874336,1.361915,-0.213059,-0.330381,-0.227903,-0.341968,-0.669889,-0.796957,-0.827230,0.0


In [5]:
df_trainVal_withTar = normalized_df.iloc[:-125,:]
df_test_withTar = normalized_df.iloc[-125:,:]
df_trainVal = normalized_df.iloc[:-125,:-1]
df_test = normalized_df.iloc[-125:,:-1]
target_df_trainVal = normalized_df.iloc[:-125,-1]
target_df_test = normalized_df.iloc[-125:,-1]

# Classification

# GenLinCFA

In [10]:
for eps in [0.3,0.4,0.5]:
    res = []
    nums = []
    for curr_seed in [0,1,2,3,4]:
        curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
        curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
        curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal), axis=1)
        
        output = GenLinCFA(curr_df_trainVal_withTar,'mean_std', eps, -5 , 0, 1).compute_clusters()
        
        aggregate_trainVal = pd.DataFrame()
        aggregate_test = pd.DataFrame()
        for i in range(len(output)):
            aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
            aggregate_trainVal = aggregate_trainVal.copy()
            aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
            aggregate_test = aggregate_test.copy()
        print(f'Number of aggregated features: {len(output)}\n')
        mod = LogisticRegression().fit(aggregate_trainVal, df_trainVal_withTar.mean_std)
        score = mod.score(aggregate_test, df_test_withTar.mean_std)
        print(score)
        res.append(score)
        nums.append(len(output))
    compute_CI(res,5)
    compute_CI(nums,5)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414


265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514


301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
537
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20

324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
557
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24

In [11]:
for eps in [0.45]:
    res = []
    nums = []
    for curr_seed in [0,1,2,3,4]:
        curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
        curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
        curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal), axis=1)
        
        output = GenLinCFA(curr_df_trainVal_withTar,'mean_std', eps, -5 , 0, 1).compute_clusters()
        
        aggregate_trainVal = pd.DataFrame()
        aggregate_test = pd.DataFrame()
        for i in range(len(output)):
            aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
            aggregate_trainVal = aggregate_trainVal.copy()
            aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
            aggregate_test = aggregate_test.copy()
        print(f'Number of aggregated features: {len(output)}\n')
        mod = LogisticRegression().fit(aggregate_trainVal, df_trainVal_withTar.mean_std)
        score = mod.score(aggregate_test, df_test_withTar.mean_std)
        print(score)
        res.append(score)
        nums.append(len(output))
    compute_CI(res,5)
    compute_CI(nums,5)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
3

In [22]:
for eps in [0.465]:
    res = []
    nums = []
    for curr_seed in [4,8,14,21,29]:
        curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
        curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
        curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal), axis=1)
        
        output = GenLinCFA(curr_df_trainVal_withTar,'mean_std', eps, -5 , 0, 1).compute_clusters()
        
        aggregate_trainVal = pd.DataFrame()
        aggregate_test = pd.DataFrame()
        for i in range(len(output)):
            aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
            aggregate_trainVal = aggregate_trainVal.copy()
            aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
            aggregate_test = aggregate_test.copy()
        print(f'Number of aggregated features: {len(output)}\n')
        mod = LogisticRegression().fit(aggregate_trainVal, df_trainVal_withTar.mean_std)
        score = mod.score(aggregate_test, df_test_withTar.mean_std)
        print(score)
        res.append(score)
        nums.append(len(output))
    compute_CI(res,5)
    compute_CI(nums,5)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308

In [ ]:
0.4:
0.7984 +- 0.011219694683902928
53.4 +- 5.8564327708939

0.45:
0.7647999999999999 +- 0.013005896016807158
26.4 +- 1.188994869627283

0.465:
0.8016 +- 0.006870631761344774
23.4 +- 1.0518463766159012

0.475:
0.7807999999999999 +- 0.007151180266221801
20.2 +- 1.5080541104350333

0.5:
0.7712 +- 0.009511958957018273
14.2 +- 1.2882434552521507

# Baselines

## pca

In [8]:
# explain 95% of variance
from sklearn.decomposition import PCA
from sklearn.utils import resample

acc = []
n = []
for curr_seed in [0,1,2,3,4]: 
    curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
    curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
    pca = PCA(n_components=0.95)
    trainVal_pca = pd.DataFrame(pca.fit_transform(curr_df_trainVal))
    test_pca = pd.DataFrame(pca.transform(df_test))
    mod = LogisticRegression().fit(trainVal_pca, curr_df_trainVal_withTar.mean_std)
    score = mod.score(test_pca, df_test_withTar.mean_std)
    acc.append(score)
    print(test_pca.shape[1])
    n.append(test_pca.shape[1])
compute_CI(acc,5)
compute_CI(n,5)
    

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

72
85
79
77
75
0.784 +- 0.01173383756492309
77.6 +- 3.8247631037751866


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
0.784 +- 0.01173383756492309
77.6 +- 3.8247631037751866

## kernel pca

In [9]:
from sklearn.decomposition import KernelPCA
for curr_kernel in ['linear', 'poly', 'sigmoid']:
    best_scores = []
    best_nums = []
    for curr_seed in [0,1,2,3,4]: 
        best_score = 0
        best_num = 0
        print(f'#########################{curr_kernel}, {curr_seed}#############################')
        for i in range(50):
            curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
            curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
            dimRedMethod = KernelPCA(n_components=i+1, kernel=curr_kernel)
            trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal))
            test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
            print(test_reduced.shape)
            mod = LogisticRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
            actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
            print(actual_score)
            if actual_score> best_score:
                best_score=actual_score
                best_num = test_reduced.shape
        print(best_score)
        print(best_num)
        best_scores.append(best_score)
        best_nums.append(best_num[1])
    compute_CI(best_scores,5)
    compute_CI(best_nums,5)

#########################linear, 0#############################
(125, 1)
0.664
(125, 2)
0.68
(125, 3)
0.72
(125, 4)
0.704
(125, 5)
0.696
(125, 6)
0.696
(125, 7)
0.712
(125, 8)
0.744
(125, 9)
0.752
(125, 10)
0.76
(125, 11)
0.76
(125, 12)
0.744
(125, 13)
0.744
(125, 14)
0.744
(125, 15)
0.76
(125, 16)
0.76
(125, 17)
0.752
(125, 18)
0.752
(125, 19)
0.752
(125, 20)
0.744
(125, 21)
0.752
(125, 22)
0.76
(125, 23)
0.752
(125, 24)
0.752
(125, 25)
0.768
(125, 26)
0.76
(125, 27)
0.76
(125, 28)
0.776
(125, 29)
0.768
(125, 30)
0.784
(125, 31)
0.768
(125, 32)
0.768
(125, 33)
0.768
(125, 34)
0.768
(125, 35)
0.768
(125, 36)
0.768
(125, 37)
0.776
(125, 38)
0.776
(125, 39)
0.784
(125, 40)
0.8
(125, 41)
0.784
(125, 42)
0.784
(125, 43)
0.784
(125, 44)
0.792
(125, 45)
0.784
(125, 46)
0.8
(125, 47)
0.784
(125, 48)
0.784


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

(125, 49)
0.792
(125, 50)
0.792
0.8
(125, 40)
#########################linear, 1#############################
(125, 1)
0.712
(125, 2)
0.736
(125, 3)
0.712
(125, 4)
0.704
(125, 5)
0.696
(125, 6)
0.76
(125, 7)
0.76
(125, 8)
0.76
(125, 9)
0.744
(125, 10)
0.76
(125, 11)
0.752
(125, 12)
0.752
(125, 13)
0.744
(125, 14)
0.752
(125, 15)
0.76
(125, 16)
0.752
(125, 17)
0.76
(125, 18)
0.76
(125, 19)
0.768
(125, 20)
0.768
(125, 21)
0.76
(125, 22)
0.752
(125, 23)
0.752
(125, 24)
0.744
(125, 25)
0.752
(125, 26)
0.744
(125, 27)
0.736
(125, 28)
0.736
(125, 29)
0.72
(125, 30)
0.736
(125, 31)
0.72
(125, 32)
0.72
(125, 33)
0.72
(125, 34)
0.736
(125, 35)
0.704
(125, 36)
0.712
(125, 37)
0.712
(125, 38)
0.712
(125, 39)
0.72
(125, 40)
0.728
(125, 41)
0.736
(125, 42)
0.736
(125, 43)
0.728
(125, 44)
0.728
(125, 45)
0.728
(125, 46)
0.728
(125, 47)
0.736
(125, 48)
0.736
(125, 49)
0.744
(125, 50)
0.72
0.768
(125, 19)
#########################linear, 2#############################
(125, 1)
0.704
(125, 2)
0.72
(125

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

(125, 6)
0.728
(125, 7)
0.728
(125, 8)
0.736
(125, 9)
0.728
(125, 10)
0.728
(125, 11)
0.728
(125, 12)
0.776
(125, 13)
0.768
(125, 14)
0.776
(125, 15)
0.76
(125, 16)
0.752
(125, 17)
0.736
(125, 18)
0.736
(125, 19)
0.752
(125, 20)
0.784
(125, 21)
0.792
(125, 22)
0.776
(125, 23)
0.768
(125, 24)
0.752
(125, 25)
0.76
(125, 26)
0.76
(125, 27)
0.76
(125, 28)
0.752
(125, 29)
0.808
(125, 30)
0.776
(125, 31)
0.776
(125, 32)
0.776
(125, 33)
0.768
(125, 34)
0.768
(125, 35)
0.776
(125, 36)
0.784
(125, 37)
0.792
(125, 38)
0.768
(125, 39)
0.768
(125, 40)
0.776
(125, 41)
0.744
(125, 42)
0.768
(125, 43)
0.76
(125, 44)
0.776
(125, 45)
0.76
(125, 46)
0.768
(125, 47)
0.768
(125, 48)
0.768
(125, 49)
0.752
(125, 50)
0.768
0.808
(125, 29)
#########################linear, 4#############################
(125, 1)
0.568
(125, 2)
0.712
(125, 3)
0.728
(125, 4)
0.72
(125, 5)
0.712
(125, 6)
0.728


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

(125, 7)
0.744
(125, 8)
0.744
(125, 9)
0.736
(125, 10)
0.744
(125, 11)
0.752
(125, 12)
0.752
(125, 13)
0.752
(125, 14)
0.768
(125, 15)
0.76
(125, 16)
0.744
(125, 17)
0.76
(125, 18)
0.752
(125, 19)
0.744
(125, 20)
0.744
(125, 21)
0.744
(125, 22)
0.736
(125, 23)
0.744
(125, 24)
0.752
(125, 25)
0.768
(125, 26)
0.752
(125, 27)
0.752
(125, 28)
0.768
(125, 29)
0.768
(125, 30)
0.768
(125, 31)
0.768
(125, 32)
0.768
(125, 33)
0.784
(125, 34)
0.792
(125, 35)
0.776
(125, 36)
0.776
(125, 37)
0.76
(125, 38)
0.776
(125, 39)
0.76
(125, 40)
0.76
(125, 41)
0.776
(125, 42)
0.792
(125, 43)
0.792
(125, 44)
0.816
(125, 45)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

0.8
(125, 46)
0.808
(125, 47)
0.8
(125, 48)
0.8
(125, 49)
0.792
(125, 50)
0.792
0.816
(125, 44)
0.792 +- 0.0177398949264081
32.4 +- 7.733435769436506
#########################poly, 0#############################
(125, 1)
0.456
(125, 2)
0.456
(125, 3)
0.608
(125, 4)
0.552
(125, 5)
0.688
(125, 6)
0.728
(125, 7)
0.728
(125, 8)
0.736
(125, 9)
0.736
(125, 10)
0.712
(125, 11)
0.704
(125, 12)
0.704
(125, 13)
0.704
(125, 14)
0.704
(125, 15)
0.736
(125, 16)
0.752
(125, 17)
0.744
(125, 18)
0.744
(125, 19)
0.744
(125, 20)
0.76
(125, 21)
0.76
(125, 22)
0.768
(125, 23)
0.784
(125, 24)
0.784
(125, 25)
0.776
(125, 26)
0.776
(125, 27)
0.784
(125, 28)
0.784
(125, 29)
0.784
(125, 30)
0.76
(125, 31)
0.784
(125, 32)
0.776
(125, 33)
0.776
(125, 34)
0.776
(125, 35)
0.776
(125, 36)
0.776
(125, 37)
0.776
(125, 38)
0.776
(125, 39)
0.776
(125, 40)
0.776
(125, 41)
0.776
(125, 42)
0.776
(125, 43)
0.776
(125, 44)
0.776
(125, 45)
0.776
(125, 46)
0.776
(125, 47)
0.784
(125, 48)
0.784
(125, 49)
0.784
(125, 50)
0.784


(125, 19)
0.736
(125, 20)
0.736
(125, 21)
0.736
(125, 22)
0.736
(125, 23)
0.736
(125, 24)
0.744
(125, 25)
0.744
(125, 26)
0.736
(125, 27)
0.736
(125, 28)
0.736
(125, 29)
0.736
(125, 30)
0.736
(125, 31)
0.728
(125, 32)
0.728
(125, 33)
0.736
(125, 34)
0.728
(125, 35)
0.728
(125, 36)
0.736
(125, 37)
0.736
(125, 38)
0.744
(125, 39)
0.744
(125, 40)
0.736
(125, 41)
0.736
(125, 42)
0.736
(125, 43)
0.736
(125, 44)
0.744
(125, 45)
0.736
(125, 46)
0.744
(125, 47)
0.744
(125, 48)
0.744
(125, 49)
0.744
(125, 50)
0.744
0.744
(125, 8)
0.7455999999999999 +- 0.012064432883480277
15.2 +- 8.945848825013755


In [ ]:
0.792 +- 0.0177398949264081
32.4 +- 7.733435769436506

0.7824 +- 0.009302879382212807
24.2 +- 3.435315880672402

0.7455999999999999 +- 0.012064432883480277
15.2 +- 8.945848825013755

## isomap

In [24]:
from sklearn.manifold import Isomap
best_scores = []
best_nums = []
for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = Isomap(n_components=i+1,n_neighbors=10)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape[1]
    print(best_score)
    print(best_num)
    best_scores.append(best_score)
    best_nums.append(best_num)
compute_CI(best_scores,5)
compute_CI(best_nums,5)

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/pao

(125, 1)
0.704
(125, 2)
0.688
(125, 3)
0.696
(125, 4)
0.744
(125, 5)
0.752
(125, 6)
0.76


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/pao

(125, 7)
0.784
(125, 8)
0.736
(125, 9)
0.752
(125, 10)
0.76
(125, 11)
0.736
(125, 12)
0.736
(125, 13)
0.744
(125, 14)
0.736
(125, 15)
0.76
(125, 16)
0.728
(125, 17)
0.72


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/pao

...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/sit

(125, 40)
0.712
(125, 41)
0.68
(125, 42)
0.68
(125, 43)
0.704


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/sit

(125, 44)
0.728
(125, 45)
0.728
(125, 46)
0.728
(125, 47)
0.688
(125, 48)
0.64
(125, 49)
0.64
(125, 50)
0.648
0.768
15
0.7856 +- 0.011219694683902954
14.8 +- 9.349015092511081


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/sit

## LLE

In [25]:
from sklearn.manifold import LocallyLinearEmbedding as LLE

best_scores = []
best_nums = []
for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = LLE(n_components=i+1,n_neighbors=10)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape[1]
    print(best_score)
    print(best_num)
    best_scores.append(best_score)
    best_nums.append(best_num)
compute_CI(best_scores,5)
compute_CI(best_nums,5)

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature nam

(125, 1)
0.576
(125, 2)
0.56
(125, 3)
0.584
(125, 4)
0.568
(125, 5)
0.696
(125, 6)
0.736
(125, 7)
0.736
(125, 8)
0.72


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature nam

(125, 9)
0.72
(125, 10)
0.72
(125, 11)
0.72
(125, 12)
0.752
(125, 13)
0.768
(125, 14)
0.76


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature nam

...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature nam

(125, 37)
0.768
(125, 38)
0.768
(125, 39)
0.768
(125, 40)
0.768
(125, 41)
0.768
(125, 42)
0.768


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature nam

(125, 43)
0.768
(125, 44)
0.768
(125, 45)
0.76
(125, 46)
0.76
(125, 47)
0.76
(125, 48)
0.752
(125, 49)
0.752
(125, 50)
0.752
0.776
32
0.7696000000000002 +- 0.05255022813271126
39.6 +- 4.6249355454968235


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


## Sup PCA

In [36]:
for kernel in ['sigmoid']: #['sigmoid']:#['poly']:#['linear']:
    best_scores = []
    best_nums = []
    for curr_seed in [1,2,3,4,5]: 
        print(f'#########################{kernel}, {curr_seed}#############################')
        best_score = 0
        best_num = 0
        for i in range(45):
            curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
            curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
            dimRedMethod = spca(num_components=i+1, kernel=kernel, degree=3, gamma=None, coef0=1)
            trainVal_reduced = pd.DataFrame(dimRedMethod.fit_and_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
            test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
            print(test_reduced.shape)
            mod = LogisticRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
            actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
            print(actual_score)
            if actual_score> best_score:
                best_score=actual_score
                best_num = test_reduced.shape
        print(best_score)
        print(best_num)
        best_scores.append(best_score)
        best_nums.append(best_num[1])
    compute_CI(best_scores,5)
    compute_CI(best_nums,5)

#########################sigmoid, 1#############################
(125, 1)
0.728
(125, 2)
0.728
(125, 3)
0.768
(125, 4)
0.736
(125, 5)
0.76
(125, 6)
0.784
(125, 7)
0.776
(125, 8)
0.728
(125, 9)
0.752
(125, 10)
0.728
(125, 11)
0.792
(125, 12)
0.752
(125, 13)
0.728
(125, 14)
0.752
(125, 15)
0.736
(125, 16)
0.76
(125, 17)
0.808
(125, 18)
0.736
(125, 19)
0.744
(125, 20)
0.784
(125, 21)
0.76
(125, 22)
0.744
(125, 23)
0.728
(125, 24)
0.744
(125, 25)
0.736
(125, 26)
0.728
(125, 27)
0.76
(125, 28)
0.736
(125, 29)
0.776
(125, 30)
0.792
(125, 31)
0.816
(125, 32)
0.768
(125, 33)
0.744
(125, 34)
0.768
(125, 35)
0.784
(125, 36)
0.736
(125, 37)
0.752
(125, 38)
0.728
(125, 39)
0.72
(125, 40)
0.728
(125, 41)
0.728
(125, 42)
0.704
(125, 43)
0.704
(125, 44)
0.712
(125, 45)
0.704
0.816
(125, 31)
#########################sigmoid, 2#############################
(125, 1)
0.728
(125, 2)
0.712
(125, 3)
0.728
(125, 4)
0.72
(125, 5)
0.728
(125, 6)
0.736
(125, 7)
0.76
(125, 8)
0.752
(125, 9)
0.744
(125, 10)
0.752

IndexError: boolean index did not match indexed array along dimension 1; dimension is 38 but corresponding boolean dimension is 31

## NCA

In [26]:
from sklearn.neighbors import NeighborhoodComponentsAnalysis as NCA

best_scores = []
best_nums = []
for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = NCA(n_components=i+1)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape
    print(best_score)
    print(best_num)
    best_scores.append(best_score)
    best_nums.append(best_num[1])
compute_CI(best_scores,5)
compute_CI(best_nums,5)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(125, 1)
0.752
(125, 2)
0.712
(125, 3)
0.72
(125, 4)
0.72


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(

(125, 5)
0.72
(125, 6)
0.744
(125, 7)
0.72
(125, 8)
0.728


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(

...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exampl

(125, 45)
0.824
(125, 46)
0.792


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exampl

(125, 47)
0.776
(125, 48)
0.816
(125, 49)
0.784
(125, 50)
0.808
0.824
(125, 45)
0.7952 +- 0.018069456748889812
28.2 +- 15.266877716154012


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## lda

In [23]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
acc = []
for curr_seed in [0,1,2,3,4]: 
    for i in range(1):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = LDA(n_components=i+1)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal,curr_df_trainVal_withTar.mean_std))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        acc.append(mod.score(test_reduced, df_test_withTar.mean_std))
    print(acc)
compute_CI(acc,5)
        

(125, 1)
[0.72]
(125, 1)
[0.72, 0.768]
(125, 1)
[0.72, 0.768, 0.704]
(125, 1)
[0.72, 0.768, 0.704, 0.76]
(125, 1)
[0.72, 0.768, 0.704, 0.76, 0.864]
0.7632 +- 0.04890551530451346


## UMAP

In [ ]:
import umap

best_scores = []
best_nums = []
for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = umap.UMAP(n_components=i+1)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape[1]
    print(best_score)
    print(best_num)
    best_scores.append(best_score)
    best_nums.append(best_num)
compute_CI(best_scores,5)
compute_CI(best_nums,5)

(125, 1)
0.632
(125, 2)
0.712
(125, 3)
0.72
(125, 4)
0.712
(125, 5)
0.736
(125, 6)
0.72
(125, 7)
0.72
(125, 8)
0.704
(125, 9)
0.728
(125, 10)
0.728
(125, 11)
0.728
(125, 12)
0.712
(125, 13)
0.72
(125, 14)
0.728
(125, 15)
0.728
(125, 16)
0.72
(125, 17)
0.728
(125, 18)
0.712
(125, 19)
0.728
(125, 20)
0.728
(125, 21)
0.728
(125, 22)
0.728
(125, 23)
0.728
(125, 24)
0.72
(125, 25)
0.728
(125, 26)
0.72
(125, 27)
0.712
(125, 28)
0.728
(125, 29)
0.72
(125, 30)
0.728
(125, 31)
0.728
(125, 32)
0.728
(125, 33)
0.728
(125, 34)
0.728
(125, 35)
0.728
(125, 36)
0.728
(125, 37)
0.728
(125, 38)
0.728
(125, 39)
0.728
(125, 40)
0.728
(125, 41)
0.728
(125, 42)
0.728
(125, 43)
0.728
(125, 44)
0.728
(125, 45)
0.728
(125, 46)
0.728
(125, 47)
0.728
(125, 48)
0.72
(125, 49)
0.72
(125, 50)
0.728
0.736
5
(125, 1)
0.68
(125, 2)
0.696
(125, 3)
0.64
(125, 4)
0.688
(125, 5)
0.688
(125, 6)
0.696
(125, 7)
0.696
(125, 8)
0.696
(125, 9)
0.696
(125, 10)
0.696
